In [2]:
import spacy, coreferee
from spacy.matcher import Matcher


In [26]:
nlp = spacy.load("en_core_web_lg")
nlp.max_length = 1200000
nlp.add_pipe('coreferee')


In [10]:
text = ""
with open("./Dune.txt", encoding="utf8") as f:
  try:
    text = f.readlines()
  except Exception as e:
    print(e)
  

In [11]:
string_text = ""
for line in text:
  string_text += line +"\n"

In [27]:
doc = nlp(string_text)


In [56]:
print(doc._.coref_chains.print())


0: study(29), his(46)
1: Muad‘Dib(34), him(44)
2: Muad’Dib(71), his(73)
3: Arrakis(78), he(88), Arrakis(101), his(111), Arrakis(133)
4: Caladan(92), his(95)
5: planet(104), their(130)
6: crone(150), she(226), Her(325), her(341)
7: boy(158), him(235), he(237), his(240)
8: Paul(160), woman(209), Paul(222), woman(284)
9: pile(175), it(197)
10: boy(263), his(271), his(279), he(310), his(314), he(381), He(397), he(430), his(446), his(451), his(480), his(520), he(562), his(576), he(622), He(626)
11: Jessica(317), woman(322)
12: Paul(336), woman(377)
13: Atreides(347), their(354)
14: woman(409), She(417)
15: ovals(462), they(477)
16: woman(496), she(515)
17: woman(557), she(574)
18: Jessica(578), she(587), her(628)
19: Arrakis(647), Arrakis(666), Arrakis(698)
20: mind(659), their(688)
21: Hawat(675), his(677), Hawat(750), Hawat(788)
22: father(678), it(686)
23: Harkonnens(693), Harkonnens(724)
24: Leto(746), Leto(764)
25: Paul(801), him(815), he(833), he(851), Paul(857), he(859), He(866), Pau

In [68]:

for coref_list in doc._.coref_chains:
  for coref in coref_list:
    for coref_index in coref:
      res = doc._.coref_chains.resolve(doc[coref_index])
      coreference_resolved += res[0].text if res else doc[coref_index].text + " "
      # if doc[coref_index].pos_ != "NOUN" or doc[coref_index].pos_ != "PROPN":
      #   print(coref_index)
      #   doc._.coref_chains.resolve(doc[coref_index])
      #   break
      

In [73]:
coreference_resolved = ""
for document in doc:
  res = doc._.coref_chains.resolve(document)
  coreference_resolved += res[0].text +" " if res else document.text + " "


In [74]:
coreference_resolved[:500]

'A beginning is the time for taking the most delicate care that the balances are correct . This every sister of the Bene Gesserit knows . To begin your study of the life of Muad‘Dib , then , take care that you first place Muad‘Dib in study time : born in the 57th year of the Padishah Emperor , Shaddam IV . And take the most special care that you locate Muad’Dib in Muad’Dib place : the planet Arrakis . Do not be deceived by the fact that Arrakis was bom on Caladan and lived Caladan first fifteen y'

In [64]:
res = doc._.coref_chains.resolve(doc[44])
print(res)

[Muad‘Dib]


In [13]:
for i in range(10):
  print(doc[i].text, "-->",doc[i].dep_,"-->", doc[i].pos_)

A --> det --> DET
beginning --> nsubj --> NOUN
is --> ROOT --> AUX
the --> det --> DET
time --> attr --> NOUN
for --> prep --> ADP
taking --> pcomp --> VERB
the --> det --> DET
most --> advmod --> ADV
delicate --> amod --> ADJ


In [14]:
i=0
for token in doc:
  if token.pos_ == 'PRON' and token._.in_coref:
    for cluster in token._.coref_clusters:

      print(token.text, "-->",cluster.main.text)
    if i == 10:
      break
    else:
      i+=1 

AttributeError: [E046] Can't retrieve unregistered extension attribute 'in_coref'. Did you forget to call the `set_extension` method?

In [103]:
pattern = [
    [ {"POS": "VERB"}, {"POS": "PART", "OP": "*"}, {"POST": "ADV", "OP": "*"},{"POS": {"IN": ["NOUN", "PRON", "PROPN"]}}],
    [ {"POS": "VERB"}, {"POST": "ADJ",
                                                                   "OP": "*"}, {"POST": "ADP",
                                                                                "OP": "*"},{"POST": "ADV", "OP": "*"}, ],
]
pattern1 =  [ {"POS": "VERB"}, {"POS": "PART", "OP": "*"}, {"POST": "ADV", "OP": "*"}]


In [87]:
pattern2 = [[{"POS": {"IN": ["PROPN"]}, "OP":"+"}, {"POS": "VERB"}, {"POS": "PART"}, {"POS": "ADV"}, {"POS": {"IN": ["NOUN",  "PROPN"]}, "OP": "*"}],
    [{"POS": {"IN": ["PROPN"]}, "OP":"+"}, {"POS": "VERB"}, {"POS": "PART", "OP": "*"}, {"POS": "ADV", "OP": "*"}, {"POS": {"IN": ["PROPN","NOUN"]}, "OP":"+"}]]


In [88]:
matcher = Matcher(nlp.vocab)
matcher.add("initial",pattern2)


In [89]:
matches = matcher(doc)

In [ ]:
matches[:30]

In [90]:
len(matches)

134

In [135]:
filtered = []
nlp_sentence = spacy.load("en_core_web_lg")
nlp_sentence.max_length = 1200000
nlp_sentence.add_pipe('coreferee')
i=0
while i < len(matches):
    current_match = matches[i]
    next_match = matches[i+1] if i+1 < len(matches) else None
    print("1)", current_match)
    print("2)", next_match)
    if not next_match:
        filtered.append(current_match)
        i+=1
    else:
        string_id = nlp.vocab.strings[current_match[0]]  # Get string representation
        span = doc[current_match[1]:current_match[2]]
        next_span = doc[next_match[1]:next_match[2]] if next_match else None
        
        if current_match[1] == next_match[1]+1:
            print(span,"---",next_span)
            longest_span = span if len(span) > len(next_span) else next_span
            print(longest_span)
            filtered.append(longest_span)
            i+=2
            # print("1)", filtered)
        elif current_match[2] == next_match[2]+1:
           

            longest_span = span if len(span) > len(next_span) else next_span
            filtered.append(longest_span)
            i+=2
        else:
            filtered.append(span)
            i+=1
            # print("3)", filtered)


1) (14946488805786704348, 1443, 1446)
2) (14946488805786704348, 2671, 2674)
1) (14946488805786704348, 2671, 2674)
2) (14946488805786704348, 2938, 2941)
1) (14946488805786704348, 2938, 2941)
2) (14946488805786704348, 3925, 3928)
1) (14946488805786704348, 3925, 3928)
2) (14946488805786704348, 3924, 3928)
Gesserit sift people --- Bene Gesserit sift people
Bene Gesserit sift people
1) (14946488805786704348, 6521, 6524)
2) (14946488805786704348, 11295, 11298)
1) (14946488805786704348, 11295, 11298)
2) (14946488805786704348, 12080, 12084)
1) (14946488805786704348, 12080, 12084)
2) (14946488805786704348, 19482, 19485)
1) (14946488805786704348, 19482, 19485)
2) (14946488805786704348, 21880, 21883)
1) (14946488805786704348, 21880, 21883)
2) (14946488805786704348, 21879, 21883)
Secundus says S.S. --- Salusa Secundus says S.S.
Salusa Secundus says S.S.
1) (14946488805786704348, 26508, 26511)
2) (14946488805786704348, 26508, 26512)
1) (14946488805786704348, 26508, 26512)
2) (14946488805786704348, 

In [136]:
def longest_string_array(array):
    longest_string = ""
    for string in array:
        if len(string) > len(longest_string):
            longest_string = string
    return longest_string

In [137]:
filteredarray = []
for match in matches:
  start = match[1]
  end = match[2]
  tempArray = []
  for new_match in matches:
    if new_match[1] == start or new_match[2] == end:
      tempArray.append(doc[new_match[0]:new_match[1]])
  if len(tempArray) > 1:
    sentence = longest_string_array(tempArray)
    if sentence != "":
      filteredarray.append(sentence)
  else:
    filteredarray.append(doc[start:end])


In [139]:
len(filteredarray)

134

In [138]:
filteredarray[:20]

[Jessica flicked dust,
 Pride overcame Paul,
 Curiosity reduced Paul,
 '',
 '',
 Leto exchange Caladan,
 Jessica shook tears,
 Jessica began to recite:“I,
 Yueh watched Paul,
 '',
 '',
 '',
 '',
 '',
 Mapes returned knife,
 Instinct warned Jessica,
 '',
 '',
 Duke involves defection,
 Paul spoke matter]